# Day 5 - Async Weather API -> CSV pipeline

This notebook demonstrates:
- Calling a real weather REST API
- Extracting structured JSON data
- Saving the result to **CSV**

In [51]:
import asyncio
import httpx
import csv
from datetime import datetime

In [52]:
URL = "https://api.open-meteo.com/v1/forecast?latitude=40.71&longitude=-74.01&current_weather=true"


In [53]:
async def fetch():
    async with httpx.AsyncClient(timeout=20) as client:
        r = await client.get(URL)
        r.raise_for_status()
        return r.json()

In [54]:
async def main():
    data = await fetch()
    cw = data["current_weather"]

    row ={
        "fetched_at": datetime.utcnow().isoformat(),
        "latitide": data["latitude"],
        "longitude": data["longitude"],
        "temperature": cw.get("temperature"),
        "windspeed": cw.get("windspeed"),
        "weathercode": cw.get("weathercode"),
        "time": cw.get("time"),
    }

    with open("weather_current.csv", "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=row.keys())
        writer.writeheader()
        writer.writerow(row)

    print("Saved weather_current.csv")

In [55]:
await main()

Saved weather_current.csv


In [56]:
import pandas as pd

df = pd.read_csv("weather_current.csv")